# Backtes the three PTS Methods

In [7]:
from Helpers.TstMethods import *
from Helpers.FunctHelpers import *

import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# Donwnload data
tickers = pd.read_excel('Data/SandP500list.xlsx')
tickers = tickers[tickers['sector'] == 'Financials']
names = tickers['symbol'].tolist()

START_DATE = pd.to_datetime('2022-01-01')
END_DATE = pd.to_datetime('2024-07-01')

data = download_data(names, START_DATE, END_DATE)

[*********************100%***********************]  65 of 65 completed


5 Failed downloads:
['WLTW', 'BRK.B', 'FRC', 'PBCT', 'RE']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [9]:
# Performe rolling backtest of the Three PTS Methods

# Define the number of testing periods
num_testing_peiods = pd.to_datetime(END_DATE) - pd.to_datetime(START_DATE)
start_date, end_date = START_DATE, START_DATE + pd.DateOffset(months=18)

# define lists to store the signals and returns
dm_backtest_signals = []
dm_backtest_returns = []

coint_backtest_signals = []
coint_backtest_returns = []

copula_backtest_signals =[]
copula_backtest_returns = []

for i in range(num_testing_peiods.days):

    # Split data into training and testing
    train_start, train_end, test_start, test_end = train_test_dates(start_date)
    train_data, test_data = train_test_split(data, train_start, train_end, test_start, test_end)

    # Get Signals
    dm_signals, dm_pairs = dm_get_signals_pairs(train_data, test_data)
    dm_backtest_signals.append(dm_signals)

    coint_signals, coint_pairs = coint_get_signals_pairs(train_data, test_data) #! Changes test df
    coint_backtest_signals.append(coint_signals)

    # copula_signals = copula_get_signals(train_data, test_data) # Copula signals are not implemented yet
    # copula_backtest_signals.append(copula_signals)

    # Calculate Returns
    dm_daily_returns_pct = calculate_returns(dm_signals, dm_pairs, test_data)
    dm_backtest_returns.append(dm_daily_returns_pct)

    coint_daily_returns_pct = calculate_returns(coint_signals, coint_pairs, test_data)
    coint_backtest_returns.append(coint_daily_returns_pct)

    # copula_daily_returns_pct = calculate_returns(copula_signals, copula_pairs, test_data)
    # copula_backtest_returns.append(copula_daily_returns_pct)

    # Update Dates, Print Info and beak
    print_backtest_info(i, start_date, end_date)
    start_date, end_date = update_dates(start_date)
    if (pd.to_datetime(end_date)) == pd.to_datetime(END_DATE):
        break

Backtest Period: 1
Start Date: 2022-01-01 End Date: 2023-07-01
-------------------------------------------
Backtest Period: 2
Start Date: 2022-02-01 End Date: 2023-08-01
-------------------------------------------
Backtest Period: 3
Start Date: 2022-03-01 End Date: 2023-09-01
-------------------------------------------
Backtest Period: 4
Start Date: 2022-04-01 End Date: 2023-10-01
-------------------------------------------
Backtest Period: 5
Start Date: 2022-05-01 End Date: 2023-11-01
-------------------------------------------
Backtest Period: 6
Start Date: 2022-06-01 End Date: 2023-12-01
-------------------------------------------
Backtest Period: 7
Start Date: 2022-07-01 End Date: 2024-01-01
-------------------------------------------
Backtest Period: 8
Start Date: 2022-08-01 End Date: 2024-02-01
-------------------------------------------
Backtest Period: 9
Start Date: 2022-09-01 End Date: 2024-03-01
-------------------------------------------
Backtest Period: 10
Start Date: 2022-

In [35]:
# Example usage of the backtesting results

# each column represents the returns of a single testing period (12 mothn calubration and 6 month testing period)
# col is a touple containing the returns of the testing period as a df

for col in dm_backtest_returns:
    
    # Get start and end date of the testing period
    start_date, end_date = col[0].index[0], col[0].index[-1]
    start_date = start_date.strftime('%Y-%m')
    end_date = end_date.strftime('%Y-%m')

    # Cumulative return of a portfolio that is equally weighted for all the pairs
    cum_return = col[0].sum().sum()
    #cum_return = col[0].cumsum().iloc[-1].sum() # also works

    print(f'Cumulative returns for the testing period, from: {start_date} to: {end_date}')
    print(f'Cumulative return: {cum_return:.2f}%')
    print('-'*69)

Cumulative returns for the testing period, from: 2023-01 to: 2023-06
Cumulative return: -3.94%
---------------------------------------------------------------------
Cumulative returns for the testing period, from: 2023-02 to: 2023-08
Cumulative return: -4.16%
---------------------------------------------------------------------
Cumulative returns for the testing period, from: 2023-03 to: 2023-09
Cumulative return: -4.69%
---------------------------------------------------------------------
Cumulative returns for the testing period, from: 2023-04 to: 2023-09
Cumulative return: 1.23%
---------------------------------------------------------------------
Cumulative returns for the testing period, from: 2023-05 to: 2023-11
Cumulative return: 0.72%
---------------------------------------------------------------------
Cumulative returns for the testing period, from: 2023-06 to: 2023-12
Cumulative return: -0.40%
---------------------------------------------------------------------
Cumulative r